In [ ]:
# Itemsets
def make_set(l,u):
    # ASCII A-Z = 65-90
    set_ = []
    l = l + 65
    u = u + 65
    for i in range(l,u+1):
        set_.append(chr(i))
    return set(set_)

sets = []
for i in range(2):
    sets.append(make_set(0,4))
for _ in range(4):
    sets.append(make_set(0,2))
for _ in range(4):
    sets.append(make_set(1,3))

display(sets)

In [ ]:

def preprocess(itemset):
    if isinstance(itemset,str):
        return set(itemset)
    else:
        return itemset

def support(itemset,sets=sets):
    itemset = preprocess(itemset)
    count = 0
    for set_ in sets:
        if itemset.issubset(set_):
            count += 1
    return count/len(sets)

def confidence(left,right,sets=sets):
    left = preprocess(left)
    right = preprocess(right)
    return support(left.union(right),sets)/support(left,sets)

def lift(left,right,sets=sets):
    left = preprocess(left)
    right = preprocess(right)
    return support(left.union(right),sets)/(support(right,sets)*support(left,sets))

In [ ]:
print(support('B',sets),support('C',sets))

In [ ]:
print(confidence('A','B'),confidence('B','C'))

In [ ]:
# Iterate over all possible itemsets
for i in range(len("ABCDE")**2):
    left = set()
    for j in range(len("ABCDE")):
        if i & (1 << j):
            left.add("ABCDE"[j])
    for k in range(len("ABCDE")**2):
        right = set()
        for l in range(len("ABCDE")):
            if k & (1 << l):
                right.add("ABCDE"[l])
        if len(left) > 0 and len(right) > 0:
            lift_ = lift(left,right)
            if lift_ < 1:
                print(left,right,lift(left,right))

In [ ]:
# FP Tree
min_support = 3/14
sets = ["ABC","ABC","ABCD","ABCD","ABDF","A","A","A","A","A","BE","BE","BE","BF"]
tree_list = [[('',14)]]
new_tree_list = []
previous_sets = set()
finished = False
while not finished:
    for set_,_ in tree_list[-1]:
        for item in "ABCDEF"[len(set_):]:
            if item in set_:
                continue
            sup = support(set_ + item,sets)
            if sup >= min_support and set_ + item not in previous_sets:
                new_tree_list.append((set_ + item,int(sup*14)))
                previous_sets.add("".join(sorted(set_ + item)))
    if len(new_tree_list) == 0:
        finished = True
    else:
        # Remove permutations
        for i in range(len(new_tree_list)):
            for j in range(i+1,len(new_tree_list)):
                if set(new_tree_list[i][0]) == set(new_tree_list[j][0]):
                    new_tree_list[j] = ('',0)
        new_tree_list = [x for x in new_tree_list if x[0] != '']
        tree_list.append(new_tree_list)
        new_tree_list = []

# Render tree using graphviz
from graphviz import Digraph
dot = Digraph(comment='FP Tree')
for i in range(len(tree_list)):
    for set_,sup in tree_list[i]:
        dot.node(set_,label=set_ + " (" + str(sup) + ")")
        if i > 0:
            dot.edge(tree_list[i-1][0][0],set_)
dot.render('fp_tree.gv', view=True)